# Model selection with non-SBML models

PEtab Select is currently designed for use with PEtab v1, which only supports SBML for model specification. However, the SBML model is irrelevant to PEtab Select, which only reads the parameter table of a model's PEtab problem to determine the estimated parameters (e.g. to accurately compute criteria values).

In this notebook, we demonstrate model selection with a modeling framework that currently does not support PEtab problems (specifically, [statsmodels](https://www.statsmodels.org/stable/index.html)). Since statsmodels is a Python package, we use the Python interface of PEtab Select. Non-Python frameworks can be used with the PEtab Select command-line interface instead. Synthetic data was generated from the model
$$y(t) = 20t + 10t^2 + 5t^3 + \epsilon$$
where $\epsilon$ is drawn from the standard normal distribution and represents measurement noise. The hypothetical scenario is that you are provided $(t,y)$ data pairs and asked to identify the powers of $t$ that are evidenced by the data (i.e., that were used to generate the data). Here, we only consider linear regression models involving terms up to $t^5$, i.e., the superset model is
$$y(t) = \sum_{i=0}^5 k_i t^i,$$
where $k_i$ is the $i$th coefficient in the linear regression model.

## "True" model

We start by loading the data and demonstrating model calibration of the true model using statsmodels.

In [ ]:
# Based on the statsmodels "Ordinary Least Squares" example
# https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html

import numpy as np
import petab
import statsmodels.api as sm
from petab import MEASUREMENT, TIME

import petab_select
from petab_select.constants import (
    CANDIDATE_SPACE,
    ESTIMATE,
    TERMINATE,
    UNCALIBRATED_MODELS,
    Criterion,
)

# Data was generated with the formula (see next cell to regenerate data)
# y(t) = 20*t + 10*t^2 + 5*t^3
df = petab.get_measurement_df(
    "other_model_types_problem/petab/measurements.tsv"
)
t = df[TIME].to_numpy()
y = df[MEASUREMENT].to_numpy()

# Show statsmodels fit for the "true" model
true_T = np.column_stack([t**1, t**2, t**3])
print(sm.OLS(y, true_T).fit().summary())

In [ ]:
# Uncomment to regenerate data

# import numpy as np
# rng = np.random.default_rng(seed=0)

# # Measurement time points
# t = np.linspace(0, 10, 101)
# # The first three powers of x are used to generate the data
# T = np.column_stack([t**1, t**2, t**3])
# K = np.array([20, 10, 5])
# noise = rng.normal(size=t.size)
# y = np.dot(T, K) + noise

# # Write PEtab measurement table
# lines = ["observableId\tsimulationConditionId\ttime\tmeasurement"]
# for t_, y_ in zip(t, y, strict=True):
#     lines.append(f"y\tdefault\t{round(t_,1)}\t{round(y_,5)}")
# content = "\n".join(lines)
# with open("other_model_types_problem/petab/measurements.tsv", "w") as f:
#     f.write(content)

## Model selection

To perform model selection, we define three methods to:

1. calibrate a single model

    This is where we convert a PEtab Select model into a statsmodels model, fit the model with statsmodels, then save the log-likelihood value in the PEtab Select model.

2. perform a single iteration of a model selection method involving calibration of multiple models

    This is generic code that executes the required PEtab Select commands.

3. perform a full model selection run, involving all iterations of a model selection method

    This loads the PEtab Select problem from disk and performs all iterations of a model selection method.

In this case, the PEtab Select problem is setup to perform backward selection ([see the specification files for this example](https://github.com/PEtab-dev/petab_select/tree/main/doc/examples/other_model_types_problem/petab_select)).

In [ ]:
def calibrate_model(model: petab_select.Model, t=t, y=y):
    # Convert the PEtab Select model into a statsmodel model
    powers = [
        int(parameter_id[1:])
        for parameter_id, parameter_value in model.parameters.items()
        if parameter_value == ESTIMATE
    ]
    if not powers:
        return
    T = np.column_stack([t**i for i in powers])
    sm_model = sm.OLS(y, T)

    # Calibrate
    results = sm_model.fit()

    # Save the log-likelihood
    model.set_criterion(criterion=Criterion.LLH, value=results.llf)


def perform_iteration(
    problem: petab_select.Problem, candidate_space: petab_select.CandidateSpace
):
    # Start the iteration, request models according to the model selection method.
    iteration = petab_select.ui.start_iteration(
        problem=problem, candidate_space=candidate_space
    )

    # Calibrate each model.
    models = iteration[UNCALIBRATED_MODELS]
    for model in models:
        if all(
            parameter_value != ESTIMATE
            for parameter_value in model.parameters.values()
        ):
            del models[model.hash]
            continue
        calibrate_model(model=model)
        # Compute the criterion based on the log-likelihood computed by statsmodels.
        model.compute_criterion(criterion=problem.criterion)

    # End the iteration and return the results.
    return petab_select.ui.end_iteration(
        problem=problem,
        candidate_space=iteration[CANDIDATE_SPACE],
        calibrated_models=models,
    )


def run_model_selection():
    # Load the PEtab Select problem.
    problem = petab_select.Problem.from_yaml(
        "other_model_types_problem/petab_select/problem.yaml"
    )
    candidate_space = None

    # Perform all iterations of the model selection method until the termination signal is emitted.
    while True:
        iteration_results = perform_iteration(
            problem=problem, candidate_space=candidate_space
        )
        if iteration_results[TERMINATE]:
            break
        candidate_space = iteration_results[CANDIDATE_SPACE]

    # Return the problem, which contains the calibrated models.
    return problem

In [ ]:
# Run the model selection method.
problem = run_model_selection()

## Analysis

As when using a PEtab-compatible calibration tool, the results from calibration with statsmodels can also be analyzed with PEtab Select.

Firstly, we get the best model identified during model selection. It matches the "true" model, with the same AIC as computed by statsmodels at the top of this notebook.

In [ ]:
best_model = petab_select.analyze.get_best(
    models=problem.state.models, criterion=problem.criterion
)
print(f"""
Best model information
----------------------
PEtab Select model hash: {best_model.hash}
Selected powers of t: {[int(parameter_id[1:]) for parameter_id, parameter_value in best_model.parameters.items() if parameter_value == ESTIMATE]}
Log-Likelihood: {best_model.get_criterion(criterion=Criterion.LLH)}
AIC: {best_model.get_criterion(criterion=Criterion.AIC)}
""")

In [ ]:
# Plot AIC of all models that were calibrated.

import petab_select.plot

plot_data = petab_select.plot.PlotData(
    models=problem.state.models, criterion=problem.criterion
)

petab_select.plot.scatter_criterion_vs_n_estimated(plot_data=plot_data);

In [ ]:
# Plot the history of model selection iterations.
import matplotlib.pyplot as plt

draw_networkx_kwargs = {
    "arrowstyle": "-|>",
    "node_shape": "s",
    "edgecolors": "k",
}
fig, ax = plt.subplots(figsize=(20, 20))
petab_select.plot.graph_iteration_layers(
    plot_data=plot_data,
    draw_networkx_kwargs=draw_networkx_kwargs,
    ax=ax,
);